Group Project Proposal : Heart Disease
-

Introduction:
- background info
- question
- identify and describe data set

Read the data into R, clean, and wrangle the data

In [1]:
#code will go here

Methods:
- explain how you will conduct the analysis and which variables will be used

Decsribe at least one way you will visualize the result:
- idk

Expected outcomes and significance:
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?